In [0]:
try:
    %tensorflow_version 1.x
except exception:
    pass

In [0]:
from google.colab import drive
import logging
import os
import random
import sys
from collections import deque
import cv2

In [26]:
path = '/content/gdrive'
drive.mount(path)
path += '/My Drive/ARVision Data'

sys.path.insert(0, path)
sys.path.insert(0, path + '/AR3DVision')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

BATCH_SIZE = 32

tf.__version__

'1.15.2'

In [28]:
def TrainingDataLoad(rootpath):
    print(rootpath)
    dataset = []

    for (path, dir, files) in os.walk(rootpath):

        for filename in files:
            ext = os.path.splitext(filename)[-1]
            
            if ext == '.txt':
                full_filename = os.path.join(path, filename)

                print(full_filename)
                f = open(full_filename, 'r')
                totalCnt = f.readline()
                descpath = os.path.join(path, f.readline().split('/')[-1])
                print(descpath[:-1])

                if os.path.isfile(descpath[:-1]) is False:
                    continue
                
                cv_file = cv2.FileStorage(descpath[:-1], cv2.FILE_STORAGE_READ)
                
                for i in range(int(totalCnt)):
                    scene = {}

                    descname = f.readline()
                    objCenter = f.readline()
                    objRot = f.readline()

                    objCenter = np.fromstring(objCenter[1:-2], sep=', ')
                    objRot = np.fromstring(objRot[1:-2], sep=', ')

                    scene['objCenter'] = objCenter;
                    scene['objRot'] = objRot
                    scene['kpts'] = []

                    kpts_cnt = int(f.readline())

                    for j in range(kpts_cnt):
                        kpt = f.readline()
                        kpt = np.fromstring(kpt[1:-2], sep=', ')

                        scene['kpts'].append(kpt)

                    if len(scene['kpts']) == 0:
                        continue

                    matrix = cv_file.getNode(descname[:-1]).mat()
                    assert matrix is not None, 'matrix read fail'

                    scene['desc'] = matrix

                    assert len(matrix) == len(scene['kpts']), 'kpt != desc'

                    dataset.append(scene)

                f.close()
                cv_file.release()
    return dataset

rootpath = "\"" + path + '/Training SURF' + "\""

print(rootpath)
!ls $rootpath
dataset = TrainingDataLoad(path + '/Training SURF')
baseset = TrainingDataLoad(path + '/Base SURF')

print('loaded data count : ' + str(len(dataset)))
print('base data count: ' +str(len(baseset)))

"/content/gdrive/My Drive/ARVision Data/Training SURF"
33900005-92F9-4917-9029-85E8D94BC5A4_dataset.txt
33900005-92F9-4917-9029-85E8D94BC5A4_dataset.xml
3DFFA0E4-21F2-4548-AE89-FE1E75D7BECA_dataset.txt
3DFFA0E4-21F2-4548-AE89-FE1E75D7BECA_dataset.xml
D5FD74A1-69E8-40D7-9509-DFF617BBEA41_dataset.txt
D5FD74A1-69E8-40D7-9509-DFF617BBEA41_dataset.xml
E8AE583F-F9BD-4B50-AB19-3873A6D881C8_dataset.txt
E8AE583F-F9BD-4B50-AB19-3873A6D881C8_dataset.xml
FD6D2A37-8BF7-47A8-84F8-B5453750BBD7_dataset.txt
FD6D2A37-8BF7-47A8-84F8-B5453750BBD7_dataset.xml
/content/gdrive/My Drive/ARVision Data/Training SURF
/content/gdrive/My Drive/ARVision Data/Training SURF/3DFFA0E4-21F2-4548-AE89-FE1E75D7BECA_dataset.txt
/content/gdrive/My Drive/ARVision Data/Training SURF/3DFFA0E4-21F2-4548-AE89-FE1E75D7BECA_dataset.xml
/content/gdrive/My Drive/ARVision Data/Training SURF/33900005-92F9-4917-9029-85E8D94BC5A4_dataset.txt
/content/gdrive/My Drive/ARVision Data/Training SURF/33900005-92F9-4917-9029-85E8D94BC5A4_datase

In [29]:
# Validate base data count
random.shuffle(baseset)
base = baseset[0]
print()
other_baseset = baseset[1:]

same_cnt = 0
total_cnt = 0;
same_pair = []
for s in other_baseset:
    data_pair = []
    for tidx, t in enumerate(zip(s['kpts'], s['desc'])):
        min_dist = sys.float_info.max
        min_desc_dist = sys.float_info.max
        kpt_idx = -1
        desc_idx = -1

        for bidx, b in enumerate(zip(base['kpts'], base['desc'])):
            dist = np.linalg.norm(b[0] - t[0])
            desc_dist = np.linalg.norm(b[1] - t[1])

            if dist < min_dist:
                min_dist = dist
                kpt_idx = bidx

            if desc_dist < min_desc_dist:
                min_desc_dist = desc_dist
                desc_idx = bidx

        total_cnt += 1
        if kpt_idx == desc_idx:
            same_cnt += 1
            data_pair.append(kpt_idx)

    # print(data_pair)

    same_pair.append(data_pair)

print(same_cnt, total_cnt)
print('total scene : ', len(baseset))
result = Counter([t for p in same_pair for t in np.unique(p)])
sorted_result = sorted(result.items(), key=lambda x:x[1], reverse=True)


355 504
total scene :  22


In [0]:
RATIO = 0.3
basedesc_idx = [idx for idx in sorted_result[:int(len(sorted_result)*RATIO)]]
basedesc = [base['desc'][i[0]] for i in basedesc_idx]
# basedesc

# Refine using base desc

In [0]:
def refineDataset(data, desc):
    refined = []
    for s in data:
        refined_data = {}
        refined_data['label'] = np.concatenate((s['objCenter'], s['objRot']), axis=None)
        
        target_idx_list = []
        for d in desc:
            target_idx = -1
            min_desc_dist = sys.float_info.max
            for idx, tdesc in enumerate(s['desc']):
                dist = np.linalg.norm(d - tdesc)
                if dist < min_desc_dist:
                    min_desc_dist = dist
                    target_idx = idx

            target_idx_list.append(target_idx)

        target_kpts = [s['kpts'][i] for i in target_idx_list]
        refined_data['kpts'] = target_kpts
        refined_data['input'] = np.concatenate(target_kpts, axis=None)

        refined.append(refined_data)

    return refined

train_data = refineDataset(dataset, basedesc)
base_data = refineDataset(baseset, basedesc)

# Train using DNN

In [0]:
from sklearn.preprocessing import StandardScaler
lr = 0.0005
epoch = 50000

scalerX = StandardScaler()
scalarY = StandardScaler()

In [0]:
X_base = [s['input'] for s in base_data]
Y_base = [s['label'] for s in base_data]

X_train = [s['input'] for s in train_data]
Y_train = [s['label'] for s in train_data]

# scalerX.fit(X_train)
# scalarY.fit(Y_train)

In [0]:
# scaled_X_base = scalerX.transform(X_base)
# scaled_Y_base = scalarY.transform(Y_base)
# scaled_X_train = scalerX.transform(X_train)
# scaled_Y_train = scalarY.transform(Y_train)

scaled_X_train = X_train
scaled_Y_train = Y_train

# len(scaled_X_train[0])

In [0]:
def model(input):
    l1 = tf.contrib.layers.fully_connected(input, 64)
    l2 = tf.contrib.layers.fully_connected(l1, 64)
    l3 = tf.contrib.layers.fully_connected(l2, 64)
    l4 = tf.contrib.layers.fully_connected(l3, 32)
    output = tf.contrib.layers.fully_connected(l4, 6, activation_fn=None)

    return output

In [0]:
input = tf.placeholder(tf.float32, [None] + [len(scaled_X_train[0])])
output = tf.placeholder(tf.float32, [None] + [6])

sess = tf.Session()

net = model(input)

In [0]:
cost = tf.reduce_mean(tf.square(net - output))
train_op = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(cost)

In [0]:
sess.run(tf.global_variables_initializer())

In [39]:
t = tqdm(range(epoch))
loss_print = ""

for i in t:
    t.refresh()
    _, loss_val = sess.run([train_op, cost], feed_dict={input: scaled_X_train, output: scaled_Y_train})
    loss_print = 'Step: [' + str(i) + '/' + str(epoch) + '], loss: ' + str(loss_val)
    t.set_description(loss_print)

Step: [49999/50000], loss: 0.39977956: 100%|██████████| 50000/50000 [09:50<00:00, 84.67it/s]


In [40]:
saver = tf.train.Saver()
save_dir = path + '/checkpoints_SURF'
save_path = os.path.join(save_dir, 'model.ckpt')
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
save_path = saver.save(sess, save_path)
tf.train.write_graph(sess.graph, save_dir, 'model.pbtxt')

'/content/gdrive/My Drive/ARVision Data/checkpoints_SURF/model.pbtxt'

In [0]:
from tensorflow.python.tools.freeze_graph import freeze_graph

In [42]:
# Provide these to run freeze_graph:
# Graph definition file, stored as protobuf TEXT
graph_def_file = save_dir + '/model.pbtxt'
# Trained model's checkpoint name
checkpoint_file = save_dir + '/model.ckpt'
# Frozen model's output name
frozen_model_file = save_dir + '/frozen_model.pb'
# Output nodes. If there're multiple output ops, use comma separated string, e.g. "out1,out2".
output_node_names = 'fully_connected_4/BiasAdd' 


# Call freeze graph
freeaed = freeze_graph(input_graph=graph_def_file,
                       input_saver="",
                       input_binary=False,
                       input_checkpoint=checkpoint_file,
                       output_node_names=output_node_names,
                       restore_op_name="save/restore_all",
                       filename_tensor_name="save/Const:0",
                       output_graph=frozen_model_file,
                       clear_devices=True,
                       initializer_nodes="")

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/ARVision Data/checkpoints_SURF/model.ckpt
INFO:tensorflow:Froze 10 variables.
INFO:tensorflow:Converted 10 variables to const ops.


In [0]:
result = sess.run([net], feed_dict={input: scaled_X_train})

In [44]:
len(result[0])

8500

In [45]:
print(result[0][1])
print(scaled_Y_train[1])

[ 0.00237682 -0.03259513  0.92192256  0.5855063   1.9214065   0.03133401]
[-0.00936609 -0.113049    1.19158     0.896794    2.49086     0.209192  ]


# Store Base desc

In [0]:
cv_file = cv2.FileStorage(save_dir + "/base_desc.xml", cv2.FILE_STORAGE_WRITE)
cv_file.write("base_desc", np.array(basedesc))
cv_file.release()